In [10]:
import os
#import pickle as pkl

#import numpy as np
import pandas as pd

In [11]:
DIR_SUBM   = os.path.join(os.getcwd(), 'subm')

In [12]:
subm_views = pd.read_csv(os.path.join(DIR_SUBM, '1_xgb_baseline_test.csv'),     usecols=['document_id', 'views'])
subm_depth = pd.read_csv(os.path.join(DIR_SUBM, '3_lgb_ttl_emb_depth_frp.csv'), usecols=['document_id', 'depth'])
subm_frp   = pd.read_csv(os.path.join(DIR_SUBM, '3_lgb_ttl_emb_depth_frp.csv'), usecols=['document_id', 'full_reads_percent'])

In [16]:
subm = subm_views.merge(subm_depth, on='document_id', validate='one_to_one')
#subm = subm.merge(subm_depth, on='document_id', validate='one_to_one')
subm = subm.merge(subm_frp, on='document_id', validate='one_to_one')

In [17]:
subm

,document_id,views,depth,full_reads_percent
0,61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,12979.607,1.162884,41.849092
1,628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,16094.666,1.050658,34.876173
2,627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,15869.438,1.061083,29.770480
3,628618629a7947d4927eb812upfii3whSSuMXCqcqF8VbQ,12979.607,1.079164,37.248951
4,620e76109a7947235623695b5hzCiIHdSYKQIr8WAM18bw,92693.090,1.197063,30.528346
...,...,...,...,...
2995,623ba6bc9a79470214cb9418x4OZQ1IjTb27nPAgv5wAZg,16735.950,1.130764,30.248436
2996,624fda1f9a79471a9bd102eetuRTrLPGSZyNbc9_YcBAeQ,42088.550,1.113462,32.456153
2997,6257a6a99a79477eea75b86dtBr3UG9ORW-gOybkVKX6ww,12979.607,1.063972,39.389542
2998,624866d59a7947421716ab46WsGDKyGjTJGRh14eR7UVvw,12979.607,1.158419,40.702991


In [19]:
subm.to_csv(os.path.join(DIR_SUBM, '3__concatenate.csv'), index = False)